In [3]:
import sys
import datetime as dt
from pathlib import Path

import pandas as pd
import numpy as np

sys.path.append('../invoicing')
from main import *

%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Read a timesheet

path = 'timesheet.csv'
timesheet = read_timesheet(path, input_time_units='min')
timesheet

,date,project,duration,comment
0,2016-04-01,A,4.000000,yeah nah
1,2016-04-04,B,2.000000,NaN
2,2016-04-04,C,1.083333,NaN
3,2016-04-06,A,10.000000,NaN
4,2016-04-13,A,5.333333,booyah
5,2016-04-14,D,12.233333,chur
6,2016-04-15,A,8.350000,NaN


In [6]:
# Summarize project durations

agg_by_project(timesheet, freq='W')


,start_date,end_date,project,duration
0,2016-03-28,2016-04-03,A,4.000000
1,2016-04-04,2016-04-10,A,10.000000
3,2016-04-04,2016-04-10,B,2.000000
4,2016-04-04,2016-04-10,C,1.083333
2,2016-04-11,2016-04-17,A,13.683333
5,2016-04-11,2016-04-17,D,12.233333


In [7]:
# Create a linear biller

base_fee = 3
biller1 = build_linear_biller(rate=30, base_fee=base_fee, freq='M')
print(biller1.__dict__)

costs = compute_costs(timesheet, biller1)
costs



{'bins': [inf], 'freq': 'M', 'rates': [30], 'kind': 'linear', 'name': None, 'base_fee': 3}


,start_date,end_date,duration,rate,cost
0,2016-04-01,2016-04-30,0.0,inf,3.0
1,2016-04-01,2016-04-30,43.0,30.000000,1290.0


In [8]:
# Create a piecewise linear biller (tiered rates)

bins = [10, 5, 7, np.inf]
print(decompose(17, bins))

rates = [30, 60, 90, 180]
biller2 = build_piecewise_linear_biller(bins, rates, base_fee, freq='W')
print(biller2.__dict__)

costs = compute_costs(timesheet, biller2)
costs

[10, 5, 2, 0]
{'bins': [10, 5, 7, inf], 'freq': 'W', 'rates': [30, 60, 90, 180], 'kind': 'piecewise_linear', 'name': None, 'base_fee': 3}


,start_date,end_date,duration,rate,cost
0,2016-03-28,2016-04-03,0.000000,inf,3.0
1,2016-03-28,2016-04-03,4.000000,30.000000,120.0
5,2016-04-04,2016-04-10,0.000000,inf,3.0
6,2016-04-04,2016-04-10,10.000000,30.000000,300.0
7,2016-04-04,2016-04-10,3.083333,60.000000,185.0
10,2016-04-11,2016-04-17,0.000000,inf,3.0
11,2016-04-11,2016-04-17,10.000000,30.000000,300.0
12,2016-04-11,2016-04-17,5.000000,60.000000,300.0
13,2016-04-11,2016-04-17,7.000000,90.000000,630.0
14,2016-04-11,2016-04-17,3.916667,180.000000,705.0


In [9]:
# Make an HTML invoice

# 1. Create a costs data frame from the template. 
# Done above.
print(costs)

# 2. Choose an HTML invoice template. 
# I'll use the example one, but you should make one suited to your needs.
template_path = Path('../examples/template/invoice.html')

# 3. Create the context for the template
context = {
  'invoice_code': '#my-first-invoice',
  'issue_date': '2016-04-30',
  'due_date': '2016-05-01',
  'currency': 'NZD',
  'sender': 'Bumptious Q. Bangwhistle',
  'sender_address': '1 Queen St, Auckland 1001',
  'sender_email': 'hello@bangwhistle.net',
  'sender_bank_name': 'Bingo Bank',
  'sender_bank_account': '1234-56-789',
  'receiver': 'Wingnut',
  'receiver_address': '2 Victoria St, Auckland 1001',
  }

# 4. Render the HTML
html = make_invoice(costs, template_path, context)

# 5. Write to file
output_path = template_path.parent / 'output.html'
with output_path.open( 'w') as tgt:
    tgt.write(html)
    

   start_date   end_date   duration        rate   cost
0  2016-03-28 2016-04-03   0.000000         inf    3.0
1  2016-03-28 2016-04-03   4.000000   30.000000  120.0
5  2016-04-04 2016-04-10   0.000000         inf    3.0
6  2016-04-04 2016-04-10  10.000000   30.000000  300.0
7  2016-04-04 2016-04-10   3.083333   60.000000  185.0
10 2016-04-11 2016-04-17   0.000000         inf    3.0
11 2016-04-11 2016-04-17  10.000000   30.000000  300.0
12 2016-04-11 2016-04-17   5.000000   60.000000  300.0
13 2016-04-11 2016-04-17   7.000000   90.000000  630.0
14 2016-04-11 2016-04-17   3.916667  180.000000  705.0
